In [1]:
#Install pre-requisists
#!pip install transformers sentencepiece sacremoses
##!git clone https://github.com/PaddlePaddle/PaddleOCR.git
#pip install paddlepaddle-gpu
#!pip install paddleocr
#!wget http://archive.ubuntu.com/ubuntu/pool/main/o/openssl/libssl1.1_1.1.0g-2ubuntu4_amd64.deb
#!sudo dpkg -i libssl1.1_1.1.0g-2ubuntu4_amd64.deb
#!pip install paddlepaddle
#!pip install "paddleocr>=2.0.1"
#!pip install paddlepaddle paddleocr

In [2]:
#Import
#from google.colab import drive
import os
import sys
#Image caption
##################################################
## import packages
##################################################

from transformers import GPT2TokenizerFast, ViTImageProcessor, VisionEncoderDecoderModel
from torch.utils.data import Dataset
from torchtext.data import get_tokenizer
import requests
import torch
import numpy as np
from PIL import Image
import pickle
# from torchvision import transforms
# from datasets import load_dataset
# import torch.nn as nn
import matplotlib.pyplot as plt
import os
from tqdm import tqdm

import warnings
warnings.filterwarnings('ignore')

# Visual Question Answering
from transformers import pipeline


# Question Augment
from transformers import *


# Random Crop Images
import cv2


# OCR
from collections import defaultdict
import numpy as np
import math
from functools import cmp_to_key
from torch._C import NoneType
from ast import dump
from paddleocr import PaddleOCR





from glob import glob
from tqdm import tqdm
import json
from matplotlib import pyplot as plt
import numpy as np


# LLM
from transformers import pipeline

#BLEW Score and Accuracy

ImportError: cannot import name '_log_class_usage' from 'torchtext.utils' (/home/xim/anaconda3/envs/ratnabali/lib/python3.8/site-packages/torchtext/utils.py)

In [ ]:
# Mount Drive
#drive.mount('/content/drive', force_remount=True)
os.chdir('/content/drive/MyDrive/gadapter/')
sys.path.insert(0,'/content/drive/MyDrive/gadapter/')

In [ ]:
VQA_baseline = pipeline("visual-question-answering", model="MariaK/vilt_finetuned_200")

Augment_baseline = PegasusForConditionalGeneration.from_pretrained("tuner007/pegasus_paraphrase")
Augment_tockenizer = PegasusTokenizerFast.from_pretrained("tuner007/pegasus_paraphrase")

Description_baseline = VisionEncoderDecoderModel.from_pretrained("nlpconnect/vit-gpt2-image-captioning")
image_processor = ViTImageProcessor.from_pretrained("nlpconnect/vit-gpt2-image-captioning")
Description_tokenizer  = GPT2TokenizerFast.from_pretrained("nlpconnect/vit-gpt2-image-captioning")

OCR_baseline = PaddleOCR(use_angle_cls=True, lang='en')


# LLM for QA
LLM_QA_baseline = pipeline('question-answering')

In [ ]:
# Functions
def M_Augment(sentence,model=Augment_baseline, tokenizer=Augment_tockenizer, num_return_sequences=5, num_beams=5):
  # tokenize the text to be form of a list of token IDs
  inputs = tokenizer([sentence], truncation=True, padding="longest", return_tensors="pt")
  # generate the paraphrased sentences
  outputs = model.generate(
    **inputs,
    num_beams=num_beams,
    num_return_sequences=num_return_sequences,
  )
  # decode the generated sentences using the tokenizer to get them back to text
  return tokenizer.batch_decode(outputs, skip_special_tokens=True)


def M_Description(path, greedy = True, model = Description_baseline):
    image = Image.open(path)
    pixel_values   = image_processor(image, return_tensors ="pt").pixel_values
    #plt.imshow(np.asarray(image))
    #plt.show()

    if greedy:
        generated_ids  = model.generate(pixel_values, max_new_tokens = 30)
    else:
        generated_ids  = model.generate(
            pixel_values,
            do_sample=True,
            max_new_tokens = 30,
            top_k=5)
    generated_text = Description_tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
    return generated_text


def get_random_crop(image, crop_height, crop_width):

    max_x = image.shape[1] - crop_width
    max_y = image.shape[0] - crop_height

    x = np.random.randint(0, max_x)
    y = np.random.randint(0, max_y)

    crop = image[y: y + crop_height, x: x + crop_width]
    cv2.imwrite('temp.jpeg',crop)
    return crop


def M_QA(I,augmented_questions):

    #image = cv2.imread(I)
    #W,H,C=image.shape

    #answers=set()
    #answers=[]
    AQA=""
    i=0
    for q in augmented_questions:
      #img=get_random_crop(image,W-50,H-50)
      #answer=pipe('temp.jpeg', q, top_k=2)
      answer=VQA_baseline(I, q, top_k=2)
      print(answer)
      AQA=AQA+ " " + answer[0]['answer']+", "
      #AQA=AQA+ q + " " + answer[0]['answer']+". "
      #answers.add(answer[0]['answer'])
      #if(i==0):
      #  AQA=AQA + q
      #  i=1
      #else:
      #  AQA=AQA+ " or " + q
      #image1 = Image.open('temp.jpeg')
      #plt.imshow(np.asarray(image1))
      #plt.show()

    #QA="If the questions are like " + AQA +" Answers are "
    #for qi in answers:
     #   QA=QA  +qi + ' or '
    #A=QA[:-3]
    return AQA

def M_OCR(I):
  result = OCR_baseline.ocr(I)
  concat_output=""
  if(np.size(result[0])>1):
    concat_output = ". ".join(row[1][0] for row in result[0])
  image_text=concat_output
  return image_text


In [ ]:
from nltk.translate.bleu_score import sentence_bleu
def CALCULATE_BLUE(answer,gt):
  candidate = answer.split()
  blew1=sentence_bleu([GT_answer.split()] , candidate, weights=(1, 0, 0, 0))
  blew2=sentence_bleu([GT_answer.split()] , candidate, weights=(0, 1, 0, 0))
  blew3=sentence_bleu([GT_answer.split()] , candidate, weights=(0, 0, 1, 0))
  blew4=sentence_bleu([GT_answer.split()] , candidate, weights=(0, 0, 0, 1))
  score = sentence_bleu([GT_answer.split()], candidate, weights=(0.25, 0.25, 0.25, 0.25))

  return [blew1,blew2,blew3,blew4,score]

def G_ADAPTER(I,question):
  O_OCR=M_OCR(I)
  O_augment=M_Augment(question)
  O_QA=M_QA(I,O_augment)
  O_description=M_Description(I)
  context=O_description + ". " + O_OCR + ". " + O_QA
  O_LLM=LLM_QA_baseline(question,context)

  print(context)
  print(question)
  answer=O_LLM['answer']
  print(answer)
  print(O_augment,O_QA)
  return answer

In [ ]:
#I="/content/drive/MyDrive/vizwiz/VizWiz_train_00000012.jpg"
#Q="what is this?"
#Q="Describe the product."
#GT_answer="remote"

#I="/content/drive/MyDrive/vizwiz/vlcsnap-2024-01-30-22h23m59s771.png"
#Q="What is the use of the product?"
#Q="Describe the product."
#GT_answer="Suthol"

I="/content/drive/MyDrive/vizwiz/VizWiz_train_00000016.jpg"
Q="What is this?"
#Q="Describe the product."
GT_answer="Tea"



image = Image.open(I).convert("RGB")
plt.imshow(np.asarray(image))
plt.show()


answer=G_ADAPTER(I,Q)
[blew1,blew2,blew3,blew4,score]=CALCULATE_BLUE(answer,GT_answer)
print(blew1,blew2,blew3,blew4,score)